In [166]:
train = 'C:/Users/ISHMO_CT/Downloads/my-projects/House-price-env/dataset/train.csv'
test = 'C:/Users/ISHMO_CT/Downloads/my-projects/House-price-env/dataset/test.csv'

In [167]:
import numpy as np
import pandas as pd

In [168]:
df = pd.read_csv(train)

In [169]:
def wrangle(df):
    df.set_index('Id', inplace=True)
    # Drop columns with more than 50 %  missing values
    # threshold = 0.5
    # #missing propotion 
    # missing_propotion = df.isnull().mean()
    # # columns to drop
    # columns_to_drop = missing_propotion[missing_propotion > threshold].index
    # # print(columns_to_drop)
    # df.drop(columns=columns_to_drop, inplace=True)

    df['HouseAge'] = df['YrSold']-df['YearBuilt']
    df['RemodelAge'] = df['YrSold']-df['YearRemodAdd']
    df['TotalPorchSF'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
    
    return df
df = wrangle(df)

In [170]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,HouseAge,RemodelAge,TotalPorchSF
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,2,2008,WD,Normal,208500,5,5,61
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,NaN,0,5,2007,WD,Normal,181500,31,31,0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,0,9,2008,WD,Normal,223500,7,6,42
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,NaN,0,2,2006,WD,Abnorml,140000,91,36,307
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,0,12,2008,WD,Normal,250000,8,8,84


In [171]:
target = 'SalePrice'
X = df.drop(columns=target)
y = df[target]

In [172]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [173]:
from sklearn.metrics import mean_absolute_error
y_pred = [y_train.mean()] * len(y_train)
baselinr_mae = mean_absolute_error(y_train, y_pred)
print(baselinr_mae)

56306.770899793584


In [174]:
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
model_lr = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    LinearRegression()
)

In [175]:
# fit the model 
model_lr.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['MSZoning', 'Street', 'Alley', 'LotShape',
                                     'LandContour', 'Utilities', 'LotConfig',
                                     'LandSlope', 'Neighborhood', 'Condition1',
                                     'Condition2', 'BldgType', 'HouseStyle',
                                     'RoofStyle', 'RoofMatl', 'Exterior1st',
                                     'Exterior2nd', 'MasVnrType', 'ExterQual',
                                     'ExterCond', 'Foundation', 'BsmtQual',
                                     'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
                                     'BsmtFinType2', 'Heating', 'HeatingQC',
                                     'CentralAir', 'Electrical', ...],
                               use_cat_names=True)),
                ('simpleimputer', SimpleImputer()),
                ('linearregression', LinearRegression())])

In [176]:
y_vall_pred = model_lr.predict(X_val)
model_lr_val_pred_mae = mean_absolute_error(y_val, y_vall_pred)
print(model_lr_val_pred_mae)

21120.223529323568


In [177]:
# make predictions 
test_df =pd.read_csv(test)
test_df = wrangle(test_df)

In [178]:
print(test_df.shape)
print(df.shape)

(1459, 82)
(1460, 83)


In [180]:
prediction = model_lr.predict(test_df)
prediction

array([123105.08417331, 154659.92434865, 187553.38696944, ...,
       178519.52124652, 120752.09342698, 217658.46344294])

In [181]:
submission_df = pd.DataFrame({"Id": test_df.index, "SalePrice": prediction})

In [183]:
submission_df.head()

,Id,SalePrice
0,1461,123105.084173
1,1462,154659.924349
2,1463,187553.386969
3,1464,193072.937209
4,1465,212982.769474


In [184]:
submission_df.to_csv('submission.csv')